# Geospatial Matching Optimization

This project is designed to showcase how to improve the speed of a polygon matching procedure. There are two main areas that could benefit from taking a "geospatial-native" approach:

1. Data encoding, fetching, and storage
2. Matching

We will generate synthetic polygon data and use both a non-geospatial-native and geospatial-native approach, comparing execution time for both to see how they differ.

Note: Make sure that you have PostgreSQL running.

## Imports

In [577]:
import geopandas as gpd
import pandas as pd
import shapely
import h3
import time
import helpers as h
import matplotlib.pyplot as plt
from shapely.wkt import dumps, loads
from pathlib import Path
import multiprocessing as mp

# ensure helpers is loaded correctly
import importlib
importlib.reload(h)

<module 'helpers' from '/Users/sra/files/projects/matching_optimization/helpers.py'>

## Generate Test Data

In [578]:
# Create two blobs of data
prev_month_blobs = h.generate_random_polygons(n=10000)
curr_month_blobs = h.generate_random_polygons(n=10000)

Function `generate_random_polygons` executed in 7.5430 sec, CPU: 6.20%, Memory: 95.98MB
Function `generate_random_polygons` executed in 8.4490 sec, CPU: 1.70%, Memory: 293.06MB


In [579]:
prev_month_blobs.head()

,geometry,id,geohash
8550,"POLYGON ((-98.16099 26.88411, -98.16099 26.884...",2a61233c-8349-44f1-b6c3-3b4cbf10c700,9uf0yq1t
8924,"POLYGON ((-98.05773 26.95524, -98.05773 26.955...",59b94d32-0ee0-4058-8e9c-b553bab8f9af,9uf32ehf
6293,"POLYGON ((-98.04039 26.97207, -98.04039 26.972...",eeb3e809-c013-4e4b-9c5b-45ed21897623,9uf33n1m
3196,"POLYGON ((-98.12466 26.99352, -98.12466 26.994...",6853b19a-766d-45f6-85fa-e41dc3e955e0,9uf1x45g
4445,"POLYGON ((-98.28726 27.49576, -98.28726 27.496...",461afc10-e942-4c35-94fe-60c3d1bae1df,9ufh7mt8


In [580]:
curr_month_blobs.head()

,geometry,id,geohash
1713,"POLYGON ((-97.86596 27.89635, -97.86596 27.896...",44f88b57-0213-4a48-a32d-ce5605413d63,9ufqtn2k
2602,"POLYGON ((-98.30079 27.89682, -98.30079 27.897...",fc24348b-2595-4608-aa0c-e3531ca92c70,9ufneneb
9463,"POLYGON ((-97.76967 27.90458, -97.76967 27.905...",98c73d2f-0725-499b-bc39-fa483f52ed0c,9ufqxpyt
9294,"POLYGON ((-98.24400 28.27086, -98.24400 28.271...",398ea29c-7ba3-47f4-95d3-6522382def60,9v40u6su
6879,"POLYGON ((-98.20837 28.33800, -98.20837 28.338...",be6ac089-d468-4c1e-a2d2-09d817973225,9v41jnz1


In [581]:
# make data folder
Path('data/').mkdir(exist_ok=True)

# save as geoparquet
prev_month_blobs.to_parquet('data/prev_month_blobs.parquet')
curr_month_blobs.to_parquet('data/curr_month_blobs.parquet')

In [582]:
# range_ = 10000

# fig, ax = plt.subplots(figsize=(5, 5))
# prev_month_blobs.iloc[0:range_].plot(ax=ax)
# ax.set_title(f'Viewing the first {range_} blobs')
# plt.tight_layout()

# plt.show()

In [583]:
prev_month_blobs.head()

,geometry,id,geohash
8550,"POLYGON ((-98.16099 26.88411, -98.16099 26.884...",2a61233c-8349-44f1-b6c3-3b4cbf10c700,9uf0yq1t
8924,"POLYGON ((-98.05773 26.95524, -98.05773 26.955...",59b94d32-0ee0-4058-8e9c-b553bab8f9af,9uf32ehf
6293,"POLYGON ((-98.04039 26.97207, -98.04039 26.972...",eeb3e809-c013-4e4b-9c5b-45ed21897623,9uf33n1m
3196,"POLYGON ((-98.12466 26.99352, -98.12466 26.994...",6853b19a-766d-45f6-85fa-e41dc3e955e0,9uf1x45g
4445,"POLYGON ((-98.28726 27.49576, -98.28726 27.496...",461afc10-e942-4c35-94fe-60c3d1bae1df,9ufh7mt8


## Non-optimized process

### 1. Data encoding, storage, and fetching

We will use PostgreSQL and string storage for geographic information.

#### Make copy of polygon layer and convert to non-geospatial-native string (WKT) datatype

In [584]:
# copy the gdfs
prev_month_blobs_wkt = prev_month_blobs.copy()
curr_month_blobs_wkt = curr_month_blobs.copy()

In [585]:
# Convert WKT versions to strings
dfs_to_convert = [prev_month_blobs_wkt, curr_month_blobs_wkt]
prev_month_blobs_wkt, curr_month_blobs_wkt = [h.convert_col_to_string(df) for df in dfs_to_convert]

# Check result
print(prev_month_blobs_wkt.head())

Function `convert_col_to_string` executed in 0.1498 sec, CPU: 9.90%, Memory: 49.22MB
Function `convert_col_to_string` executed in 0.0534 sec, CPU: 0.00%, Memory: 0.34MB
                                               geometry  \
8550  POLYGON ((-98.160994 26.884114, -98.160994 26....   
8924  POLYGON ((-98.057729 26.955244, -98.057729 26....   
6293  POLYGON ((-98.040389 26.972066, -98.040389 26....   
3196  POLYGON ((-98.124658 26.993519, -98.124658 26....   
4445  POLYGON ((-98.287261 27.495764, -98.287261 27....   

                                        id   geohash  
8550  2a61233c-8349-44f1-b6c3-3b4cbf10c700  9uf0yq1t  
8924  59b94d32-0ee0-4058-8e9c-b553bab8f9af  9uf32ehf  
6293  eeb3e809-c013-4e4b-9c5b-45ed21897623  9uf33n1m  
3196  6853b19a-766d-45f6-85fa-e41dc3e955e0  9uf1x45g  
4445  461afc10-e942-4c35-94fe-60c3d1bae1df  9ufh7mt8  


/Users/sra/files/projects/matching_optimization/helpers.py:305: UserWarning: Geometry column does not contain geometry.
  return df
/Users/sra/files/projects/matching_optimization/helpers.py:305: UserWarning: Geometry column does not contain geometry.
  return df


### Save in PostgreSQL database

This is a simple version of what a "non-geospatial-native" data ingestion pipeline may look like, with the primary example being that the polygons are stored as strings, not in a spatial-friendly datatype.

In [586]:
# Convert to tuples
prev_month_blobs_wkt = h.df_itertuple(prev_month_blobs_wkt)
curr_month_blobs_wkt = h.df_itertuple(curr_month_blobs_wkt)

Function `df_itertuple` executed in 0.0050 sec, CPU: 0.00%, Memory: 0.00MB
Function `df_itertuple` executed in 0.0044 sec, CPU: 0.00%, Memory: 0.00MB


Create PostgreSQL database if it doesn't exist already. We will be using the default settings. If they need to be adjusted, refer to [`helpers.py`](helpers.py).

In [587]:
h.create_pg_db()

Database blob_matching already exists.
Function `create_pg_db` executed in 0.0734 sec, CPU: 0.00%, Memory: 0.00MB


Create and insert into tables:

In [588]:
prev_month_blobs.head(1)

,geometry,id,geohash
8550,"POLYGON ((-98.16099 26.88411, -98.16099 26.884...",2a61233c-8349-44f1-b6c3-3b4cbf10c700,9uf0yq1t


In [589]:
prev_month_blobs_wkt[0]

('POLYGON ((-98.160994 26.884114, -98.160994 26.884855, -98.160456 26.884855, -98.160456 26.884114, -98.160994 26.884114))',
 '2a61233c-8349-44f1-b6c3-3b4cbf10c700',
 '9uf0yq1t')

In [590]:
h.create_pg_table(table_name='prev_blobs_wkt', data=prev_month_blobs_wkt, truncate=True)
h.create_pg_table(table_name='curr_blobs_wkt', data=curr_month_blobs_wkt, truncate=True)

Table prev_blobs_wkt truncated.
Inserted 10000 records into prev_blobs_wkt.
Function `create_pg_table` executed in 0.7394 sec, CPU: 69.50%, Memory: 17.80MB
Table curr_blobs_wkt truncated.
Inserted 10000 records into curr_blobs_wkt.
Function `create_pg_table` executed in 0.6693 sec, CPU: 27.90%, Memory: 6.95MB


Retrieve data as GeoDataFrames to confirm that it worked:

In [591]:
df_prev = h.retrieve_pg_table(table_name='prev_blobs_wkt')
df_curr = h.retrieve_pg_table(table_name='curr_blobs_wkt')

Retrieved 10000 records from prev_blobs_wkt.
Function `retrieve_pg_table` executed in 0.1258 sec, CPU: 39.40%, Memory: 9.20MB
Retrieved 10000 records from curr_blobs_wkt.
Function `retrieve_pg_table` executed in 0.0628 sec, CPU: 55.30%, Memory: 4.48MB


Compare the tables before and after for a sanity check:

In [592]:
df_prev.head()

,geometry,id,geohash
0,"POLYGON ((-98.160994 26.884114, -98.160994 26....",2a61233c-8349-44f1-b6c3-3b4cbf10c700,9uf0yq1t
1,"POLYGON ((-98.057729 26.955244, -98.057729 26....",59b94d32-0ee0-4058-8e9c-b553bab8f9af,9uf32ehf
2,"POLYGON ((-98.040389 26.972066, -98.040389 26....",eeb3e809-c013-4e4b-9c5b-45ed21897623,9uf33n1m
3,"POLYGON ((-98.124658 26.993519, -98.124658 26....",6853b19a-766d-45f6-85fa-e41dc3e955e0,9uf1x45g
4,"POLYGON ((-98.287261 27.495764, -98.287261 27....",461afc10-e942-4c35-94fe-60c3d1bae1df,9ufh7mt8


In [593]:
prev_month_blobs.head()

,geometry,id,geohash
8550,"POLYGON ((-98.16099 26.88411, -98.16099 26.884...",2a61233c-8349-44f1-b6c3-3b4cbf10c700,9uf0yq1t
8924,"POLYGON ((-98.05773 26.95524, -98.05773 26.955...",59b94d32-0ee0-4058-8e9c-b553bab8f9af,9uf32ehf
6293,"POLYGON ((-98.04039 26.97207, -98.04039 26.972...",eeb3e809-c013-4e4b-9c5b-45ed21897623,9uf33n1m
3196,"POLYGON ((-98.12466 26.99352, -98.12466 26.994...",6853b19a-766d-45f6-85fa-e41dc3e955e0,9uf1x45g
4445,"POLYGON ((-98.28726 27.49576, -98.28726 27.496...",461afc10-e942-4c35-94fe-60c3d1bae1df,9ufh7mt8


In [594]:
def round_geometry(geom, precision=6):
    """Round all coordinates of a geometry to a given precision."""
    return shapely.wkt.loads(shapely.wkt.dumps(geom, rounding_precision=precision))

# Convert both to sets of rounded WKT strings
set_prev_month_blobs = set(prev_month_blobs['geometry'].apply(lambda g: round_geometry(g, precision=6).wkt))
set_df_prev = set(df_prev['geometry'].apply(lambda g: round_geometry(g, precision=6).wkt))

# Find common, missing, and extra geometries
common_geometries = set_prev_month_blobs & set_df_prev
missing_from_retrieved = set_prev_month_blobs - set_df_prev
extra_in_retrieved = set_df_prev - set_prev_month_blobs

# Print summary
print(f"Number of matching geometries: {len(common_geometries)}")
print(f"Missing geometries in retrieved table: {len(missing_from_retrieved)}")
print(f"Extra geometries in retrieved table: {len(extra_in_retrieved)}")

# Show an example missing/extra geometry for debugging
if missing_from_retrieved:
    print("Example missing record:", next(iter(missing_from_retrieved)))

if extra_in_retrieved:
    print("Example extra record:", next(iter(extra_in_retrieved)))

Number of matching geometries: 10000
Missing geometries in retrieved table: 0
Extra geometries in retrieved table: 0


It worked!

### 2. Matching

We will match the polygons using GeoPandas.

In [595]:
# postgresql_details = h.pg_details()
# h.run_parallel_matching(
#     "prev_blobs_wkt", "curr_blobs_wkt", "matched_results",
#     postgresql_details, "blob_matching", num_workers=4, batch_size=50
# )

In [596]:
postgresql_details = h.pg_details()
h.run_parallel_matching(table_prev='prev_blobs_wkt', 
                        table_curr='curr_blobs_wkt', 
                        output_table='matched_results', 
                        postgresql_details=postgresql_details, 
                        db_name='blob_matching', 
                        num_workers=4, 
                        batch_size=100)

Retrieved 10000 records from prev_blobs_wkt.
Function `run_parallel_matching` executed in 40.2449 sec, CPU: 15.50%, Memory: 461.84MB


Baseline - Polygon Matching Only

geohash polygon into a larger region
match blobs using multiprocessing (mp.Process) and geopandas
find blobs from previous month that do not have a match

# 1 imports and setup

```python
import os, sys, traceback
import argparse
import cv2
import time
import numpy as np
import pandas as pd
import geopandas as gpd
from skimage import io, color, measure
from tqdm import tqdm
from shapely import wkt
from shapely.geometry import Point, Polygon
from shapely.validation import make_valid
import PIL
from PIL import Image, ImageDraw, ImageEnhance
import uuid
import multiprocessing as mp
from datetime import datetime
from sqlalchemy import or_
from sqlmodel import SQLModel, Session, create_engine, select
from pathlib import Path
```

## Key takeaways
- Uses Pandas and GeoPandas for data processing.
- Uses Shapely for geospatial geometry operations.
- Uses multiprocessing (mp) to parallelize the blob-matching process.
- Uses SQLAlchemy and SQLModel for database operations.

# 2 High-level script overview
The script is designed to match blobs (spatial objects) between two months and classify them into different business categories.

General Workflow
1. Fetch blobs from the previous and current months.
1. Convert the region of interest (state, city, county, or geohash) into a list of geohashes.
1. Parallelized Matching Process:
1. Process multiple geohashes at once using multiprocessing.
1. Find corresponding blobs from previous months for each geohash.
1. Classify blobs based on construction stage progression.
1. Identify blobs that are missing in the current month and "impute" them.
1. Save results into the database.

# 3 Blob classification logic

# 4 Blob matching

- Matches blobs between months using polygon intersections.
- Uses Shapely to validate geometries and check for overlaps.
- Returns matched blob IDs.

## Multiprocessing Optimization
The script parallelizes blob matching by:
- Uses multiprocessing (mp.Process) to divide the dataset into smaller batches.
- Each batch of blobs is processed in parallel.
- Reduces runtime compared to a single-threaded approach.

# 5 Handling unmatched blobs
- Finds blobs from the previous month that do not have a match in the current month.
- These blobs are "imputed", meaning they are carried over into the new month.

# 6 Main class to orchestrate process
- Loads previous and current month blobs.
- Converts input regions (state, city, county) into geohashes.

# Summary
- Parallel processing is used to match blobs.
- Blobs are classified based on construction stages.
- Unmatched blobs are imputed for continuity.
- The script writes results to a database.